In [ ]:
import numpy as np
import cv2, math, time
from scipy import optimize
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

In [ ]:
img = cv2.imread("Images\saw_01.png", 0)

ret,img_thresholded = cv2.threshold(img,127,255,cv2.THRESH_BINARY)

kernel = np.ones((3,3),np.uint8)
img_contours = img_thresholded - cv2.erode(img_thresholded, kernel)     # Contours are white, background is black
img_contours_negative = 255 - img_contours                              # Contours are black, background is white

img_cc = img_contours_negative[400:600,210:600]
plt.imshow(img_cc, cmap = 'gray')
plt.title('Contours Cropped')
plt.xticks([]), plt.yticks([])
plt.show()
cv2.imwrite("Images\saw_01_contours_cropped.png", img_cc)
raw_contour = [[i,j] for i in range(img_cc.shape[0]) for j in range(img_cc.shape[1]) if img_cc[i,j] == 0]

In [ ]:
## Gif of the contours being drawn, order of the points drawn is the order of the list raw_contours (from top to bottom)
#blank = np.ones((img_cc.shape[0], img_cc.shape[1]), np.uint8) * 255
#for i in raw_contours:
#    blank[i[0], i[1]] = 0
#    cv2.imshow('image', blank)
#    cv2.waitKey(1)
#
#cv2.waitKey(0)    
#cv2.destroyAllWindows()

In [ ]:
def sort_contour_points(points):
    sorted_points = sorted(points, key=lambda p: p[1])
    start_point = sorted_points[0]
    sorted_points.remove(start_point)
    sorted_contour = [start_point]

    while sorted_points:
        last_point = sorted_contour[-1]

        euclidean_distances = [((last_point[0] - p[0]) ** 2 + (last_point[1] - p[1]) ** 2) ** 0.5 for p in sorted_points]
        min_distance = min(euclidean_distances)
        nearest_neighbors = [i for i in range(len(euclidean_distances)) if euclidean_distances[i] == min_distance]

        for i in sorted(nearest_neighbors, reverse=True):
            sorted_contour.append(sorted_points[i])
            sorted_points.remove(sorted_points[i])
            
    return sorted_contour

'''
Why has been necessary to sort the points of the contour?
    - The contour is a list of points that are not ordered in any way, so the first step is to order them in a way that
    makes sense. The order of the points is important because it will be used to calculate the angle between each pair of
    points, and the angle is the most important feature of the contour.
Why has been necessary to use the euclidean distance to sort the points?
    - The euclidean distance is used to find the nearest neighbor of a point, and it is used because it is the most
    straightforward way to find the nearest neighbor of a point in a 2D space.
Why has been necessary to use the nearest neighbors?
    - Because the nearest neighbors of a point are the next points in the contour, and it is important to find the nearest
    neighbors of a point to order the contour. 
'''

sorted_contour = sort_contour_points(raw_contour)

In [ ]:
## Gif of the contours being drawn, order of the points drawn is the order of the list sorted_contour (from left to right)
#blank = np.ones((img_cc.shape[0], img_cc.shape[1]), np.uint8) * 255
#for i in sorted_contour:
#    blank[i[0], i[1]] = 0
#    cv2.imshow('image', blank)
#    cv2.waitKey(1)
#
#cv2.waitKey(0)    
#cv2.destroyAllWindows()

In [ ]:
# Implementation of RDP Algorithm (Optimized)
def RDP_Algorithm(points, epsilon):
    # get the start and end points
    start = points[0]
    end = points[-1]

    # find distance from other points to line formed by start and end
    dist_point_to_line = DPTL(points, start, end)

    # get the index of the points with the largest distance
    max_value = max(dist_point_to_line)
    max_idx = dist_point_to_line.index(max_value) + 1 #since the first (and the last point) are not included in the calculation

    result = []
    if max_value > epsilon:
        if len(points[:max_idx+1]) == 2:
            result += [list(i) for i in points[:max_idx+1] if list(i) not in result]
        else:
            partial_results_left = RDP_Algorithm(points[:max_idx+1], epsilon)
            result += [list(i) for i in partial_results_left if list(i) not in result]
        if len(points[max_idx:]) == 2:
            result += [list(i) for i in points[max_idx:] if list(i) not in result]
        else:
            partial_results_right = RDP_Algorithm(points[max_idx:], epsilon)
            result += [list(i) for i in partial_results_right if list(i) not in result]
    else:
        result += [points[0], points[-1]]
    
    return result

def DPTL(points, start, end):
    # return a list of distances: distance of each point in points to line formed by start and end

    # compute the angular coefficient and the constant of the line formed by start and end
    # y - mx - q = 0
    a = start[1] - end[1]
    b = end[0] - start[0]
    c = - a*start[0] - b*start[1] 

    return [abs(a*points[i][0]+b*points[i][1]+c)/(math.sqrt(a**2+b**2)) for i in range(1,len(points))]

In [ ]:
simplified_contour = RDP_Algorithm(sorted_contour, epsilon = 3)

def distance(p1, p2):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5


def segment_contours(points, threshold, min_points):
    segments = []
    i = 0

    while i < len(points):

        close_points = [points[i]]

        for j in range(i,len(points)-1):
            if distance(points[j], points[j+1]) < threshold:
                close_points.append(points[j+1])
            else:
                break

        if len(close_points) >= min_points:
            segments.append(close_points)
            i += len(close_points)
        else:
            segments.append([points[i]])
            i += 1

    return segments

segmented_contour = segment_contours(simplified_contour, threshold=30, min_points=4)

In [ ]:
simplified_contour = RDP_Algorithm(sorted_contour, epsilon = 3)

blank = np.ones((img_cc.shape[0], img_cc.shape[1], 3), np.uint8) * 255
for i in range(len(simplified_contour)):
    blank[simplified_contour[i][0], simplified_contour[i][1]] = [0,0,255]
cv2.imwrite("Images\saw_01_simplified_contours.png", blank)

#def line_length(point1, point2):
#    return math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)
#
#ll = [line_length(simplified_contours[i], simplified_contours[i+1]) for i in range(len(simplified_contours))]
#length_threshold = 20
#for i in ll:
#    if i < length_threshold:
#        index = ll.index(i)
#        simplified_contours.remove(simplified_contours[index])
#
#plt.plot(simplified_contours[:,1], simplified_contours[:,0], 'b')
#plt.imshow(img_cc, cmap = 'gray')
#plt.title('Simplified contours')
#plt.xticks([]), plt.yticks([])
#plt.show()

In [ ]:
# TO BE TESTED
'''
Differentiating between straight lines and circular arcs in a contour can be a challenging task. However, one approach could be 
to use the circle fitting method mentioned earlier and then measure the variance or standard deviation of the distances from the points to the fitted circle.

If the variance is small, it means that the points fit the circle well, and the segment can be considered a circular arc. If the variance is large, it means 
that the points do not fit the circle well, and the segment can be considered a straight line.
'''
def classify_segment(points, threshold):
    # Fit a circle to the points
    xc, yc, R = least_squares_circle(points)

    # Calculate the distances from the points to the circle
    distances = np.sqrt((points[:,0]-xc)**2 + (points[:,1]-yc)**2)

    # Calculate the variance of the distances
    variance = np.var(distances)

    # If the variance is small, classify the segment as a circular arc
    # If the variance is large, classify the segment as a straight line
    if variance < threshold:
        return 'circular arc'
    else:
        return 'straight line'

def calc_R(xc, yc, points):
    """ calculate the distance of each 2D points from the center (xc, yc) """
    return np.sqrt((points[:,0]-xc)**2 + (points[:,1]-yc)**2)

def f_2(c, points):
    """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
    Ri = calc_R(*c, points)
    return Ri - Ri.mean()

def least_squares_circle(points):
    # coordinates of the barycenter
    x_m = np.mean(points[:,0])
    y_m = np.mean(points[:,1])
    center_estimate = x_m, y_m
    center, _ = optimize.leastsq(f_2, center_estimate, args=(points))
    xc, yc = center
    Ri       = calc_R(*center, points)
    R        = Ri.mean()
    return xc, yc, R


segments = classify_segment(np.array(sorted_contour), 2)
print(segments)